In [13]:
import os
#os.environ['TRANSFORMERS_CACHE'] = '/home/bonna46/gollum/sadia/cache'
import torch
from huggingface_hub import login
from datasets import load_dataset
import os
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
from datasets import load_dataset
from trl import SFTTrainer

In [14]:
#Importing the dataset
dataset_name='/home/bonna46/envSadia/FEN_DATABASE_train_30k_updated.csv'

In [15]:
dataset = load_dataset('csv', data_files=dataset_name, split="train[0:15000]")
dataset['FEN'][200]
#dataset1=load_dataset('csv', data_files=dataset_name, split="train[850:950]")
#print(len(dataset))


'r1bk1bnr/2pn2pp/p2p1p2/1N1qp1B1/Pp6/2NP1P2/1PPKP1PP/1R1Q1B1R w - - 9 13'

In [16]:
import pandas as pd
df = pd.DataFrame(dataset)
#df1=pd.DataFrame(dataset1)
# Method 1: Using the '+' operator
df['train_input'] = df['instruction_fen_nl'] + " ### Instruction: "+ df['FEN_and_Languages_ver'] +" ### Response: "+  df['FEN_output']
df['test_input']= df['instruction_fen_nl'] + " ### Instruction: "+ df['FEN_and_Languages_ver'] + " ### Response: "

print(df['train_input'][3])

You are a chess grandmaster. You understand and can locate chess pieces given the FEN state and the description of current state of the chess board. Your task is to consider factors such as material advantage, positional strength, and potential tactics like forks, pins, and discovered attacks to find the next move that produce the best centipawn improvement for the active player. You will describe the move by giving a starting square of piece and ending square as coordinates of board . Take extra care to only use legal and valid move. ### Instruction: Here is the FEN state: rnb5/3pbp2/3k1n1r/pp4pp/2P2Pq1/K1NpB2N/PP2P1BP/R5R1 w - - 0 21. And here is the language version of the corresponding FEN state: The current state of the chess board is: Black rook at a8, black knight at b8 and black bishop at c8. Black pawn at d7, black bishop at e7 and black pawn at f7. Black king at d6, black knight at f6 and black rook at h6. Black pawn at a5, black pawn at b5, black pawn at g5 and black pawn at

In [17]:
from datasets import Dataset
testDataset = Dataset.from_pandas(df)
print(testDataset)

Dataset({
    features: ['Instruction_fen', 'FEN_input', 'FEN_output', 'instruction_fen_nl', 'FEN_and_Languages_ver', 'FEN', 'output', 'FEN + Next move', 'instruction_nl', 'NL Format of FEN', 'Output for NLP', 'NL Format of FEN + Next move', 'train_input', 'test_input'],
    num_rows: 15000
})


In [18]:
print('hf running')
device = 'cuda' if torch.cuda.is_available() else 'cpu'     
print(f'Using device: {device}')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# with huggingface
#login(token='Your Token')
base_model = "meta-llama/Meta-Llama-3-70B"

hf running
Using device: cuda
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/bonna46/.cache/huggingface/token
Login successful


In [19]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)


model.config.use_cache = False
model.config.pretraining_tp = 1



Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

In [20]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token
print(model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 8192)
    (layers): ModuleList(
      (0-79): 80 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=8192, out_features=8192, bias=False)
          (k_proj): Linear4bit(in_features=8192, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=8192, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=8192, out_features=8192, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=8192, out_features=28672, bias=False)
          (up_proj): Linear4bit(in_features=8192, out_features=28672, bias=False)
          (down_proj): Linear4bit(in_features=28672, out_features=8192, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

In [21]:
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense',
        'fc1',
        'fc2',
    ]
)
model = get_peft_model(model, peft_config)
model.get_memory_footprint()


42814459904

In [22]:
training_arguments = TrainingArguments(
    output_dir="/home/Llama-3-70B-15k-outputDir",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    save_strategy="steps",
    save_steps=200,
    logging_steps=100,
    logging_strategy="steps",
    learning_rate=5e-5,
    fp16=False,
    bf16=False,
    group_by_length=True,
    disable_tqdm=False,
    report_to="none",
)

In [23]:
trainer = SFTTrainer(
    model=model,
    train_dataset=testDataset,
    peft_config=peft_config,
    max_seq_length= 1024,
    dataset_text_field="train_input",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/home/bonna46/envSadia/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/bonna46/envSadia/lib/python3.10/site-packages/transformers/training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/home/bonna46/envSadia/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/bonna46/envSadia/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to th

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

In [25]:
trainer.train()
#trainer.train(resume_from_checkpoint = True)

/home/bonna46/envSadia/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
trainer.save_model("/home/Llama-3-70B-15k-outputDir")
trainer.tokenizer.save_pretrained("/home/Llama-3-70B-15k-outputDir")